# 基于 OrangePi 的多模态韩语 VQA 微调全流程

本 Notebook 整合了从数据准备到模型导出的完整微调流程。基于 OpenDataLab “万卷·丝路” 韩语数据，使用 LLaMA-Factory 对 Qwen2-VL 模型进行 LoRA 微调。

> **注意**：本流程不包含推理测试部分，专注于模型的训练与导出。

## 📋 流程目录
1. **环境准备**：依赖安装与路径配置
2. **数据处理**：
   - 图片下载与索引构建
   - 损坏图片清洗
   - ShareGPT 格式转换
   - 训练/验证集分割
3. **数据集注册**：配置 LLaMA-Factory
4. **模型微调**：启动 LoRA 训练
5. **模型导出**：合并权重并导出

## 1. 环境准备与配置

首先定义工程目录结构，并安装必要的依赖库。

In [ ]:
# === 安装依赖 ===
!git clone https://github.com/hiyouga/LLaMA-Factory.git
!pip install -e LLaMA-Factory/[metrics]
!pip install modelscope requests tqdm pillow


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### 需下载”书生万卷“韩语图文标注数据集并且放到工作目录中
https://opendatalab.com/OpenDataLab/WanJuanSiLu2O/blob/main/raw/image/ko/ko_image_caption.jsonl

In [3]:
import os
import json
import random
import shutil
import hashlib
import argparse
import requests
import concurrent.futures as futures
from pathlib import Path
from typing import Optional, Tuple
from urllib.parse import urlparse
from requests.adapters import HTTPAdapter, Retry
from PIL import Image
# 修改后的写法（纯文本进度条，兼容性好）
from tqdm import tqdm

# === 全局路径配置 ===
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, "data")
IMAGE_ROOT = os.path.join(DATA_DIR, "images", "ko")

# 输入文件 (请确保此文件已上传到工作目录)
RAW_DATA_PATH = "ko_image_caption.jsonl"

# 中间产物
CLEAN_JSON_PATH = os.path.join(DATA_DIR, "ko_caption_clean.json")
SHAREGPT_PATH = os.path.join(DATA_DIR, "ko_sharegpt.json")

# 最终数据集
TRAIN_FILE = os.path.join(DATA_DIR, "ko_train.json")
EVAL_FILE = os.path.join(DATA_DIR, "ko_eval.json")

# 模型相关
MODEL_NAME = "Qwen/Qwen2-VL-2B-Instruct"
OUTPUT_DIR = "saves/Qwen2-VL/lora/Qwen2-VL-sft-ko"
EXPORT_DIR = "models/Qwen2-VL-sft-final"

# 创建必要目录
os.makedirs(IMAGE_ROOT, exist_ok=True)
print(f"工作目录已就绪: {BASE_DIR}")

工作目录已就绪: e:\Desktop\workspace\orange-pi-mindspore\Online\community\KoreanVQA


## 2. 数据处理：图片下载

读取原始 JSONL，多线程下载图片到本地，过滤掉不可下载的文件

可选择下载的数据集数量，替换路径。

In [4]:
# === 配置下载参数 ===
MAX_DOWNLOAD_LINES = 2000  # 限制处理行数，0为处理全部
NUM_WORKERS = 32           # 下载并发数

# === 核心下载逻辑 ===
FORMAT_EXT_MAP = {
    "JPEG": ".jpg", "JPG": ".jpg", "PNG": ".png",
    "WEBP": ".webp", "BMP": ".bmp", "GIF": ".gif",
}

def guess_ext(url: str, fmt: Optional[str]) -> str:
    path = urlparse(url).path
    suf = Path(path).suffix.lower()
    if suf in {".jpg", ".jpeg", ".png", ".webp", ".bmp", ".gif"}:
        return ".jpg" if suf == ".jpeg" else suf
    if fmt:
        ext = FORMAT_EXT_MAP.get(fmt.upper())
        if ext: return ext
    return ".jpg"

def build_local_path(out_dir: Path, img_id: Optional[str], url: str, fmt: Optional[str]) -> Path:
    ext = guess_ext(url, fmt)
    if img_id and isinstance(img_id, str) and len(img_id) >= 2:
        key = img_id
    else:
        key = hashlib.sha1(url.encode("utf-8")).hexdigest()
    subdir = key[:2]  # 使用前两位哈希分桶
    return out_dir / subdir / f"{key}{ext}"

def make_session() -> requests.Session:
    sess = requests.Session()
    retries = Retry(total=3, backoff_factor=0.5, status_forcelist=[429, 500, 502, 503, 504])
    adapter = HTTPAdapter(max_retries=retries, pool_connections=64, pool_maxsize=64)
    sess.mount("http://", adapter)
    sess.mount("https://", adapter)
    return sess

def process_line(line: str, out_dir: Path) -> Tuple[bool, Optional[str]]:
    try:
        rec = json.loads(line)
    except: return False, None

    image = rec.get("image", {}) or {}
    url = image.get("path")
    if not isinstance(url, str) or not url.startswith("http"):
        return False, None

    local_path = build_local_path(out_dir, rec.get("img_id"), url, image.get("format"))
    local_path.parent.mkdir(parents=True, exist_ok=True)

    # 如果文件已存在且非空，跳过下载
    if not (local_path.exists() and local_path.stat().st_size > 0):
        try:
            session = make_session()
            with session.get(url, stream=True, timeout=5) as resp:
                resp.raise_for_status()
                with open(local_path, "wb") as f:
                    for chunk in resp.iter_content(chunk_size=8192):
                        f.write(chunk)
        except Exception:
            if local_path.exists(): local_path.unlink()
            return False, None

    image["path"] = str(local_path.as_posix())
    rec["image"] = image
    return True, json.dumps(rec, ensure_ascii=False)

# === 执行下载任务 ===
print(f"开始处理图片，源文件: {RAW_DATA_PATH}")
if not os.path.exists(RAW_DATA_PATH):
    print("❌ 错误：找不到源文件，请上传 ko_image_caption.jsonl")
else:
    lines = []
    with open(RAW_DATA_PATH, "r", encoding="utf-8") as f:
        if MAX_DOWNLOAD_LINES > 0:
            for i, line in enumerate(f):
                if i >= MAX_DOWNLOAD_LINES: break
                lines.append(line.strip())
        else:
            lines = [l.strip() for l in f]

    success_cnt = 0
    with open(CLEAN_JSON_PATH, "w", encoding="utf-8") as fw:
        with futures.ThreadPoolExecutor(max_workers=NUM_WORKERS) as ex:
            tasks = [ex.submit(process_line, line, Path(IMAGE_ROOT)) for line in lines]
            for fut in tqdm(futures.as_completed(tasks), total=len(tasks), desc="Downloading"):
                ok, res = fut.result()
                if ok and res:
                    fw.write(res + "\n")
                    success_cnt += 1
    
    print(f"✅ 下载完成。成功: {success_cnt}/{len(lines)}。结果已保存至: {CLEAN_JSON_PATH}")

开始处理图片，源文件: ko_image_caption.jsonl


Downloading: 100%|██████████| 2000/2000 [16:15<00:00,  2.05it/s] 

✅ 下载完成。成功: 635/2000。结果已保存至: e:\Desktop\workspace\orange-pi-mindspore\Online\community\KoreanVQA\data\ko_caption_clean.json


## 3. 数据处理：清洗与格式转换

清洗坏图并且将数据转换为shareGPT格式
这里将 Prompt 统一设置为韩语：**"이 사진을 설명해주세요."** (请描述这张图片)。

shareGPT样例：  

       {
         "messages": [
           {"role": "user", "content": "<image>그들은 누구입니까?"},
           {"role": "assistant", "content": "그들은 바이에른 뮌헨의 케인과 고레츠카입니다."},
         ],
         "images": ["demo_data/1.jpg"]
       }

In [5]:
def verify_image(path):
    try:
        with Image.open(path) as img:
            img.verify()
        return True
    except:
        return False

print("开始格式转换与坏图清洗...")
sharegpt_data = []
valid_count = 0

if os.path.exists(CLEAN_JSON_PATH):
    with open(CLEAN_JSON_PATH, "r", encoding="utf-8") as f:
        for line in tqdm(f, desc="Processing"):
            try:
                data = json.loads(line)
                img_path = data.get("image", {}).get("path")
                caption = data.get("captions", {}).get("content")
                
                # 校验图片
                if not img_path or not os.path.exists(img_path) or not verify_image(img_path):
                    continue
                
                # 构建 ShareGPT 格式 (韩语 Prompt)
                entry = {
                    "messages": [
                        {"role": "user", "content": "<image>이 사진을 설명해주세요."}, 
                        {"role": "assistant", "content": caption}
                    ],
                    "images": [img_path]
                }
                sharegpt_data.append(entry)
                valid_count += 1
            except Exception as e:
                continue
    
    # 保存结果
    with open(SHAREGPT_PATH, "w", encoding="utf-8") as f:
        json.dump(sharegpt_data, f, ensure_ascii=False, indent=2)
    
    print(f"✅ 转换完成。有效数据: {valid_count} 条。保存至: {SHAREGPT_PATH}")
else:
    print("❌ 未找到下载后的数据文件，请先运行步骤 2。")

开始格式转换与坏图清洗...


Processing: 635it [00:11, 56.51it/s]

✅ 转换完成。有效数据: 570 条。保存至: e:\Desktop\workspace\orange-pi-mindspore\Online\community\KoreanVQA\data\ko_sharegpt.json


## 4. 数据处理：数据集分割

将数据划分为训练集与验证集。

In [ ]:
N_EVAL = 70  # 验证集数量
SEED = 42

if os.path.exists(SHAREGPT_PATH):
    with open(SHAREGPT_PATH, "r", encoding="utf-8") as f:
        all_data = json.load(f)
    
    total = len(all_data)
    random.seed(SEED)
    random.shuffle(all_data)
    
    # 确保数据量足够
    if total <= N_EVAL:
        n_eval = int(total * 0.1)
    else:
        n_eval = N_EVAL
        
    n_train = total - n_eval
    train_data = all_data[:n_train]
    eval_data = all_data[n_train:]
    
    with open(TRAIN_FILE, "w", encoding="utf-8") as f:
        json.dump(train_data, f, ensure_ascii=False, indent=2)
    with open(EVAL_FILE, "w", encoding="utf-8") as f:
        json.dump(eval_data, f, ensure_ascii=False, indent=2)
        
    print(f"✅ 分割完成。\n   训练集: {len(train_data)} -> {TRAIN_FILE}\n   验证集: {len(eval_data)} -> {EVAL_FILE}")
else:
    print("❌ 找不到 ShareGPT 数据文件。")

✅ 分割完成。
   训练集: 70 -> e:\Desktop\workspace\orange-pi-mindspore\Online\community\KoreanVQA\data\ko_train.json
   验证集: 500 -> e:\Desktop\workspace\orange-pi-mindspore\Online\community\KoreanVQA\data\ko_eval.json


## 5. 下载底座模型

In [ ]:
!git lfs install
!git clone https://www.modelscope.cn/Qwen/Qwen2-VL-2B-Instruct.git models/Qwen2-VL-2B-Instruct

## 5. 注册数据集

将生成的 json 文件路径写入 LLaMA-Factory 的 `dataset_info.json` 中，以便框架识别。**

       "ko_train": {
         "path": "data/ko_train.json",
         "type": "sharegpt_multi_modal"
       },
       "ko_val": {
         "path": "data/ko_val.json",
         "type": "sharegpt_multi_modal"
       }

## 6. 模型微调 (LoRA)

使用 LLaMA-Factory CLI 启动训练。

### 关键参数示例：

   | 选项 | 值 |
   | ---- | -- |
   | Model name  | Qwen2-VL-2B-Instruct |
   | Model path  | models/Qwen2-VL-2B-Instruct |
   | Finetune    | LoRA |
   | Stage       | Supervised Fine-Tuning |
   | Dataset     | ko_train |
   | Max epochs  | 3 |
   | Batch size  | 16 |
   | Save steps  | 200 |
   | lora_rank   | 64 |
   | lora_alpha  | 128（一般是rank的两倍） |
   | lora_dropout | 0.05（防止过拟合） |
   | Output dir  | saves/Qwen2-VL/lora/Qwen2-VL-sft-ko |

### 监控显存  
       watch -n 1 nvidia-smi

## 7. 模型导出

训练完成后，将 LoRA 权重合并到底座模型中，以便在 OrangePi 上部署。

在 WebUI **Expert** 标签执行  

    Model path      = models/Qwen2-VL-2B-Instruct
    Checkpoint path = saves/Qwen2-VL/lora/Qwen2-VL-sft-ko
    Export path     = models/Qwen2-VL-sft-final

点击“开始导出”，得到合并权重。